<a href="https://colab.research.google.com/github/MatchLab-Imperial/deep-learning-course/blob/master/03_Network_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework

### Task 1: Tuning a Classification Model
In a machine learning problem, and especially when using a deep learning approach, finding the right set of hyperparameters, the right data augmentation strategy, or a good regularization method can make the difference between a model that performs poorly and a model with great accuracy.

For this exercise, you will be training a CNN to perform classification in CIFAR-10 (we use the official test set, which is why the variables are called `x_test` and `y_test`, as our validation set) and will analyze the impact of some of the most important elements presented in this tutorial.

Use the CNN we give in the code below, along with the given optimizer and number of training epochs as the default setting. Only modify the given CNN architecture to add Dropout or Batch Normalization layers when explicitly stated. Use 40 epochs to plot all of your curves. However, you can train for more epochs to find your best validation performance if your network has not finished training in those 40 epochs.

**Report:**
*  First, train the given default model without any data augmentation. Then define two data augmentation strategies (one more aggressive than the other) and train the model with data augmentation. Clearly state the two augmentation strategies you apply (i.e., the specific transformations). Discuss the training and validation loss curves for the two data augmentation strategies along with the original run without data augmentation. Attach in the appendix those training and validation curves. Report in a table the best validation accuracy obtained for the three runs (no data augmentation, data augmentation 1, data augmentation 2).

*  Without using any data augmentation, analyze the effect of using Dropout in the model. Carry out the same analysis for Batch Normalization. Finally, combine both. Report in the same table as in the data augmentation task the best validation accuracy for each of the three settings (baseline + Dropout, baseline + Batch Normalization, baseline + Batch Normalization + Dropout). The performance will vary depending on where the Dropout layers and Batch Normalization layers are, so state clearly where you added the layers, and what rate you used for the Dropout layers. Discuss the results.

* Using the default model/hyperparameters and no data augmentation, report the best validation accuracy when using `zeros` for the kernel initialization. Report the performance in the same table as in the dropout/batch normalization/data augmentation tasks. Discuss the results that you obtained.

*  Using the default model and no data augmentation, change the optimizer to SGD and train it with learning rates of `3e-3`, `1e-3` and `3e-4`. Report in a figure the training and validation loss for the three learning rate values and discuss the figure.

In [ ]:
# Loading CIFAR-10 train set to calculate mean and std for normalizations
x = datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(x, batch_size=len(x), shuffle=False)
data_tensor, _ = next(iter(loader))

# Compute mean and std per channel
mean = data_tensor.mean(dim=[0,2,3])
std = data_tensor.std(dim=[0,2,3])

mean = mean.tolist()
std = std.tolist()

print("Per-channel mean:", mean)
print("Per-channel std:", std)

In [ ]:
# Default: No augmentation
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
    # You can later add augmentations like:
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)), etc.
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

train_loader = data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

print("Image shape:", np.array(train_dataset[0][0]).shape)
print("Total number of training samples:", len(train_dataset))
print("Total number of validation samples:", len(test_dataset))

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, use_dropout=False, use_batchnorm=False, dropout_rate=0.5, init_mode="xavier_uniform"):
        super().__init__()

        def conv_block(in_channels, out_channels, use_bn, init, use_dropout, dropout_rate):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
            if init == "zeros":
                nn.init.constant_(conv.weight, 0.0)
                nn.init.constant_(conv.bias, 0.0)
            else:
                nn.init.xavier_uniform_(conv.weight)
                nn.init.constant_(conv.bias, 0.0)
            layers = [conv]
            if use_bn:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU())
            if use_dropout:
                layers.append(nn.Dropout2d(dropout_rate))
            return nn.Sequential(*layers)


        # 4 conv blocks with ReLU and MaxPool2d(padding=1)
        self.features = nn.Sequential(
            conv_block(3, 32, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(32, 64, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(64, 128, use_batchnorm, init_mode, use_dropout, dropout_rate),
            nn.MaxPool2d(2,2, padding=0),

            conv_block(128, 256, use_batchnorm, init_mode, use_dropout, dropout_rate)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1,1))

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate if use_dropout else 0.0),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.global_pool(x)
        x = self.classifier(x)
        return x

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct_train = 0
        total_train = 0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            preds = outputs.argmax(dim=1)
            correct_train += (preds == targets).sum().item()
            total_train += targets.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct_train / total_train
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        model.eval()
        val_loss = 0
        correct_val = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                correct_val += (preds == targets).sum().item()

        val_loss /= len(test_loader.dataset)
        val_acc = correct_val / len(test_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(
            f"Epoch [{epoch+1}/{epochs}] "
            f"- Train Loss: {train_loss:.4f},  Train Acc: {train_acc*100:.2f}%, "
            f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc*100:.2f}%"
        )

    history = {
        "train_loss": train_losses,
        "train_acc": train_accuracies,
        "val_loss": val_losses,
        "val_acc": val_accuracies
    }
    return history

In [ ]:
set_seed(42)

# Use "model = CNNModel()" to get the default model
model = CNNModel(use_dropout=False, use_batchnorm=False, init_mode="xavier_uniform")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4) # Edit optimizer and learning rate here.

history = train_model(model, train_loader, test_loader, criterion, optimizer, epochs=40)